In [21]:
import csv
import numpy as np

with open(r'C:\Users\Thier\Music\DS-Sprint-01-Dealing-With-Data\module4-databackedassertions\persons _clean.csv', 'rt') as f:
    data = list(csv.DictReader(f))
data[:1]

[OrderedDict([('age', '44'), ('weight', '118'), ('exercise_time', '192')])]

In [22]:
from sklearn.utils import shuffle
from sklearn.model_selection import train_test_split

_all_xs = [{k: v for k, v in row.items() if k != 'age'} for row in data]
_all_ys = np.array([int(row['age']) for row in data])

all_xs, all_ys = shuffle(_all_xs, _all_ys, random_state=0)
train_xs, valid_xs, train_ys, valid_ys = train_test_split(
    all_xs, all_ys, test_size=0.25, random_state=0)
print('{} items total, {:.1%} true'.format(len(all_xs), np.mean(all_ys)))

1200 items total, 4839.7% true


In [23]:
for x in all_xs:
    if x['exercise_time']:
        x['exercise_time'] = float(x['exercise_time'])
    else:
        x.pop('age')
    x['weight'] = float(x['weight'])

In [24]:
import warnings
# xgboost <= 0.6a2 shows a warning when used with scikit-learn 0.18+
warnings.filterwarnings('ignore', category=DeprecationWarning)
from xgboost import XGBClassifier
from sklearn.feature_extraction import DictVectorizer
from sklearn.pipeline import make_pipeline
from sklearn.model_selection import cross_val_score

class CSCTransformer:
    def transform(self, xs):
        # work around https://github.com/dmlc/xgboost/issues/1238#issuecomment-243872543
        return xs.tocsc()
    def fit(self, *args):
        return self

clf = XGBClassifier()
vec = DictVectorizer()
pipeline = make_pipeline(vec, CSCTransformer(), clf)

def evaluate(_clf):
    scores = cross_val_score(_clf, all_xs, all_ys, scoring='accuracy', cv=10)
    print('Accuracy: {:.3f} ± {:.3f}'.format(np.mean(scores), 2 * np.std(scores)))
    _clf.fit(train_xs, train_ys)  # so that parts of the original pipeline are fitted

evaluate(pipeline)

C:\Users\Thier\AppData\Local\Continuum\anaconda3\lib\site-packages\sklearn\model_selection\_split.py:626: Warning: The least populated class in y has only 8 members, which is too few. The minimum number of members in any class cannot be less than n_splits=10.
  % (min_groups, self.n_splits)), Warning)


Accuracy: 0.019 ± 0.018


In [25]:
booster = clf.get_booster()
original_feature_names = booster.feature_names
booster.feature_names = vec.get_feature_names()
print(booster.get_dump()[0])
# recover original feature names
booster.feature_names = original_feature_names

0:[exercise_time<235.5] yes=1,no=2,missing=2
	1:[weight<141.5] yes=3,no=4,missing=3
		3:[weight<134.5] yes=5,no=6,missing=5
			5:leaf=-0.0183061827
			6:leaf=0.183229133
		4:[weight<191.5] yes=7,no=8,missing=7
			7:leaf=0.00089393137
			8:leaf=-0.0421104841
	2:leaf=-0.0425097719



In [26]:
from eli5 import show_weights
show_weights(booster, vec=vec)

Weight,Feature
0.5279,exercise_time
0.4721,weight


In [27]:
from eli5 import show_prediction
show_prediction(booster, valid_xs[1], vec=vec, show_feature_values=True)

In [28]:
no_missing = lambda feature_name, feature_value: not np.isnan(feature_value)
show_prediction(booster, valid_xs[1], vec=vec, show_feature_values=True, feature_filter=no_missing)